In [29]:
import numpy as np
import os

# plotting 
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import PyPDF2

# data manager and analysis
import vodex as vx
import numan as nu

# writing files
import tifffile as tif

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Project structure: 

Provide the project folder with the "processed" folder created in the previous notebook. 

As you keep going with the analysis, the folder will have the following structure: 

```
processed
│   experiment.json <----------------------------------- (DONE in 01) the file that contains everything about the experiment, you are creating it once and will be reusing ever after
│   experiment_dff.json <------------------------------- (DONE in 01) everything about the experiment, but loads from the dff movie, not from the raw data
└───dff_movie  <---------------------------------------- (DONE in 01) the dff movie :) 
│   │   dff_movie_0000.tif
│   │   dff_movie_0001.tif
│   │   ... 
└───tscore_volumes  <----------------------------------- (DONE in 02) t-score tif files per pair
│   └───2v3
│       │   tscore_2v3.tif
│   └───3v5
│       │   tscore_3v5.tif
│   └───2v5
│       │   tscore_2v5.tif
│   └───2vB
│       │   tscore_2vB.tif
│   └───3vB
│       │   tscore_3vB.tif
│   └───5vB
│       │   tscore_5vB.tif
│   └───BvB
│       │   tscore_BvB1.tif
│       │   tscore_BvB2.tif
│       │   tscore_BvB3.tif
│       │   tscore_BvB4.tif
│       │   tscore_BvB5.tif
└───diff_volumes  <------------------------------------- (DONE in 02) absolute difference tif files per pair
│   └───2v3
│       │   diff_2v3.tif
│   └───3v5
│       │   diff_3v5.tif
│   └───...
└───spots
│   └───imaris  <--------------------------------------- (DONE after 02) ATTENTION : You need to put stuff generated by imaris into this folder!!! 
│       │   └───tscore_2v3_Statistics
│       │       │     tscore_2v3_Position.csv
│       │       │     tscore_2v3_Diameter.csv
│       │       │     ...
│       │   └───tscore_3v5_Statistics
│       │       │     tscore_3v5_Position.csv
│       │       │     tscore_3v5_Diameter.csv
│       │       │     ...
│       │   └───tscore_2v5_Statistics
│       │       │     ...
│       │   └───...
│   └───signals  <-------------------------------------- (WILL BE DONE in this notebook) json files with the extracted signals, also will have the group info after you added it
│       │   spots_2v3.json
│       │   spots_3v5.json
│       │   spots_2v5.json
│       │     ...
└───reports  <------------------------------------------ tiffs and pdf with the cells and signals found 
│   └───all_significant  <------------------------------ tiffs and pdf with all significant cells per group
│       │   └───signals  <------------------------------ pdfs with signals
│       │       │     significant_2v3.pdf
│       │       │     significant_3v5.pdf
│       │       │     ...
│       │   └───images <-- tif masks 
│       │       │     significant_2v3.tif
│       │       │     significant_3v5.tif
│       │       │     ...
│   └───groupped  <------------------------------------- tiffs and pdf where the cells are groupped based on signal shape .. or anything else you want
│       │   readme.txt  <------------------------------- ATTENTION : you need to describe the groups
│       │   └───signals  <------------------------------ pdfs with signals
│       │       │     shape1_2v3.pdf
│       │       │     shape2_2v3.pdf
│       │       │     shape3_2v3.pdf
│       │       │     ...
│       │       │     shape1_3v5.pdf
│       │       │     shape2_3v5.pdf
│       │       │     shape3_3v5.pdf
│       │       │     ...
│       │   └───images  <------------------------------ tif masks 
│       │       │     shape1_2v3.tif
│       │       │     shape2_2v3.tif
│       │       │     shape3_2v3.tif
│       │       │     ...
│       │       │     shape1_3v5.tif
│       │       │     shape2_3v5.tif
│       │       │     shape3_3v5.tif
│       │       │     ...
```

Also, if the processed folder should already exist if you created dff movies. If the folder doesn't exist, it will complain ...

# Make sure you have the imaris files in the right folders... 

You can use any way to segemnt the images, we need position and diameter


```
processed
│   
└───spots
│   └───imaris  <--------------------------------------- ATTENTION : You need to put stuff generated by imaris into this folder!!! 
│       │   └───tscore_2v3_Statistics
│       │       │     tscore_2v3_Position.csv
│       │       │     tscore_2v3_Diameter.csv
│       │       │     ...
│       │   └───tscore_3v5_Statistics
│       │       │     tscore_3v5_Position.csv
│       │       │     tscore_3v5_Diameter.csv
│       │       │     ...
│       │   └───tscore_2v5_Statistics
│       │       │     ...
│       │   └───...
└───...  
```

# Set project folder

In [31]:
project_folder = "D:/Code/repos/numan/notebooks/data/2vs3vs5/"
path = os.path.join(project_folder, 'processed')

assert os.path.isdir(path), "the directory 'processed' doesn't exist in the project," \
                                " did you forget to run the previous notebook?"

os.chdir(path)
os.getcwd()

'D:\\Code\\repos\\numan\\notebooks\\data\\2vs3vs5\\processed'

# Load experiment with the raw data and define conditions: 

In [32]:
# don't forget to give the conditions names - they are used for plotting. 
blank = vx.Condition(vx.Stimulus('blank','blank'), name = 'blank')
dot2 = vx.Condition(vx.Stimulus('2dot','2dot'), name = 'dot2')
dot3 = vx.Condition(vx.Stimulus('3dot','3dot'), name = 'dot3')
dot5 = vx.Condition(vx.Stimulus('5dot','5dot'), name = 'dot5')

experiment = vx.from_json(vx.Experiment,'experiment.json')
experiment.summary()

Total of 8 files.
Check the order :
[ 0 ] 20220421_ok08_abtl_h2bcamp6s_9dpf_2v3v5_2P_1_MMStack_Pos0.ome.tif : 8910 frames
[ 1 ] 20220421_ok08_abtl_h2bcamp6s_9dpf_2v3v5_2P_1_MMStack_Pos0_1.ome.tif : 8909 frames
[ 2 ] 20220421_ok08_abtl_h2bcamp6s_9dpf_2v3v5_2P_1_MMStack_Pos0_2.ome.tif : 8909 frames
[ 3 ] 20220421_ok08_abtl_h2bcamp6s_9dpf_2v3v5_2P_1_MMStack_Pos0_3.ome.tif : 8909 frames
[ 4 ] 20220421_ok08_abtl_h2bcamp6s_9dpf_2v3v5_2P_1_MMStack_Pos0_4.ome.tif : 8909 frames
[ 5 ] 20220421_ok08_abtl_h2bcamp6s_9dpf_2v3v5_2P_1_MMStack_Pos0_5.ome.tif : 8909 frames
[ 6 ] 20220421_ok08_abtl_h2bcamp6s_9dpf_2v3v5_2P_1_MMStack_Pos0_6.ome.tif : 8909 frames
[ 7 ] 20220421_ok08_abtl_h2bcamp6s_9dpf_2v3v5_2P_1_MMStack_Pos0_7.ome.tif : 4092 frames

Cycle length: 3692
Condition ['blank']: for 364 frames
Condition ['3dot']: for 52 frames
Condition ['blank']: for 260 frames
Condition ['2dot']: for 52 frames
Condition ['blank']: for 312 frames
Condition ['5dot']: for 52 frames
Condition ['blank']: for 312 fra

## Extract spot signals
This is done for each set of segmentations.

Make the directory for signals:

In [ ]:
path = os.path.join(path, 'spots', 'signals')
os.makedirs(path)


Get a wrapper function

In [35]:
# if you run out of memory, reduce the batch size (this is how many timepoints are being 
BATCH_SIZE = 100
RESOLUTION = [4.8, 1.17, 1.17]

def load_and_save_signals(points_file, diameter_file, group_tag):
    # Load spots from the Position and Diameter
    spots = nu.Spots.from_imaris(points_file,diameter_file,resolution = RESOLUTION, units='phs')
    # extract signal ( this takes a long time)
    spots.get_signals(volumes="all", experiment=experiment, batch_size=BATCH_SIZE, traces_type="raw")
    spots.to_json(f"spots/signals/spots_{group_tag}.json")

Extract the signals and save, sorry it outputs a lot of stuff ... 

In [36]:
tag = "2v3"
print(f'{tag}___________________________________________________________________________________________')
points_file = f"spots/imaris/tscore_{tag}_Statistics/tscore_{tag}_Position.csv"
diameter_file = f"spots/imaris/tscore_{tag}_Statistics/tscore_{tag}_Diameter.csv"
load_and_save_signals(points_file, diameter_file, tag)


tag = "3v5"
print(f'{tag}___________________________________________________________________________________________')
points_file = f"spots/imaris/tscore_{tag}_Statistics/tscore_{tag}_Position.csv"
diameter_file = f"spots/imaris/tscore_{tag}_Statistics/tscore_{tag}_Diameter.csv"
load_and_save_signals(points_file, diameter_file, tag)

tag = "2v5"
print(f'{tag}___________________________________________________________________________________________')
points_file = f"spots/imaris/tscore_{tag}_Statistics/tscore_{tag}_Position.csv"
diameter_file = f"spots/imaris/tscore_{tag}_Statistics/tscore_{tag}_Diameter.csv"
load_and_save_signals(points_file, diameter_file, tag)

tag = "2vB"
print(f'{tag}___________________________________________________________________________________________')
points_file = f"spots/imaris/tscore_{tag}_Statistics/tscore_{tag}_Position.csv"
diameter_file = f"spots/imaris/tscore_{tag}_Statistics/tscore_{tag}_Diameter.csv"
load_and_save_signals(points_file, diameter_file, tag)


tag = "3vB"
print(f'{tag}___________________________________________________________________________________________')
points_file = f"spots/imaris/tscore_{tag}_Statistics/tscore_{tag}_Position.csv"
diameter_file = f"spots/imaris/tscore_{tag}_Statistics/tscore_{tag}_Diameter.csv"
load_and_save_signals(points_file, diameter_file, tag)

tag = "5vB"
print(f'{tag}___________________________________________________________________________________________')
points_file = f"spots/imaris/tscore_{tag}_Statistics/tscore_{tag}_Position.csv"
diameter_file = f"spots/imaris/tscore_{tag}_Statistics/tscore_{tag}_Diameter.csv"
load_and_save_signals(points_file, diameter_file, tag)


2v3___________________________________________________________________________________________


100%|█████████████████████████████████████████████████████████████████████████████| 5096/5096 [00:15<00:00, 333.98it/s]


3v5___________________________________________________________________________________________


100%|█████████████████████████████████████████████████████████████████████████████| 5096/5096 [00:15<00:00, 330.49it/s]


2v5___________________________________________________________________________________________


...

100%|█████████████████████████████████████████████████████████████████████████████| 5096/5096 [00:15<00:00, 324.26it/s]


3vB___________________________________________________________________________________________


100%|█████████████████████████████████████████████████████████████████████████████| 5096/5096 [00:15<00:00, 328.52it/s]


5vB___________________________________________________________________________________________


100%|█████████████████████████████████████████████████████████████████████████████| 5096/5096 [00:15<00:00, 334.05it/s]
